## Première version de carte interactive : lit le fichier csv (séparateur = ;) qui sort de requete.py et l'affiche sur une carte (Folium)

In [1]:
import folium as fm
import csv
import pandas as pd
from folium.plugins import MarkerCluster

In [2]:
cs = 3 #nombre de chiffres significatifs

In [3]:
#création du fond de carte (on prend un point arbitraitre dans la communauté de communes de Montereau)
m = fm.Map(location=[48.39406536332271, 2.9518367606924776], zoom_start=12)

Le fichier produit par le programme requete.py se nomme DLE-elec-2019-geocoded.csv

Ses données sont au format : OPERATEUR ; ANNEE ; FILIERE ; CODE IRIS ; NOM COMMUNE ; ADRESSE ; CODE GRAND SECTEUR ; CONSO (en MWh) ; PDL ; latitude ; longitude


In [4]:
#ici on va faire des tests avec un petit fichier qui s'appelle mini_test_DLE dans le dossier code

In [5]:
base_elec = pd.read_csv("../data/DLE-elec-2019-CCPM-geocoded.csv", ";")
base_gaz = pd.read_csv("../data/DLE-gaz-2019-CCPM-geocoded.csv", ";")

In [6]:
#documentation
dico_secteur = {'A': 'Agriculture', 'I' : 'Industriel', 'X' : 'Professionnel non affecté', 'R' : 'Résidentiel', 'T' : 'Tertiaire'}
dico_filiere = {'E' : 'Electricité' , 'G' : 'Gaz'}
dico_couleur_filiere = {'E' : 'blue', 'G' : 'green' }

In [7]:
def significatifs(data,c):
    """
    convertit la chaine de caractère data
    qui est forcément de la forme X,XXXXe+XX
    en une chaine en écriture scientifique 
    avec c chiffres significatifs
    """
    new = data[:2]
    i = 0
    
    while 1+i <= c :
        new += data[2+i]
        i += 1
        
    end = 'e'+ data[-3] + data[-2] + data[-1]
    return new + end        
        

In [8]:
def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(fm.Element(script + css))

    return folium_map

In [9]:
#code couleur pour distinguer Gaz et Elec dans les DLE
m = add_categorical_legend(m, 'Filiere',
                             colors = ['#64cd1e','#03cafc'],
                           labels = ['Gaz', 'Electricité'])

In [10]:
def non_breaking_spaces(data):
    """
    remplace les espaces dans un str par des &nbsp
    &nbsp = non breaking space en HTML
    évite des retours à la ligne excessifs
    """
    new = ''
    for i in range(len(data)):
        if data[i] != ' ':
            new += data[i]
        else:
            new += '&nbsp'
    return new

In [11]:
def pourcentage(score):
    """
    écrit le score en pourcentage
    """
    fl = float(score)
    fl = 100*fl
    return str(int(fl))

In [12]:
#regroupement des points (clusters) pour éviter les amas de markers
cluster_DLE_all = MarkerCluster().add_to(m)

In [13]:
##pour l'électricité

for i in range(len(base_elec)):

    data = base_elec.loc[i]

    geocode = [data['latitude'],data['longitude']]

    name_tip = non_breaking_spaces(data['ADRESSE'])
    filiere = data['FILIERE']
    code_grand_secteur = data['CODE_GRAND_SECTEUR']
    name_grand_secteur = dico_secteur[code_grand_secteur]
    color_filiere = dico_couleur_filiere[filiere]

    score = pourcentage(data['score'])
    
    fm.Marker(geocode,
    popup=f"<p><strong>{name_tip}</strong></p><p><strong>Secteur&nbsp:&nbsp</strong>{name_grand_secteur}<p>{str(data['PDL'])}&nbsppoint(s)&nbspde&nbsplivraison</p><p><strong>Consommation</strong>: {significatifs(data['CONSO'],cs)}&nbspMWh</p><p><em>Fiable&nbspà&nbsp{score}&nbsp%</em></p>",tooltip=name_tip,icon = fm.Icon(color=color_filiere)).add_to(cluster_DLE_all)



In [14]:
##pour le gaz

for i in range(len(base_gaz)):

    data = base_gaz.loc[i]

    geocode = [data['latitude'],data['longitude']]

    name_tip = non_breaking_spaces(data['ADRESSE'])
    filiere = data['FILIERE']
    code_grand_secteur = data['CODE_GRAND_SECTEUR']
    name_grand_secteur = dico_secteur[code_grand_secteur]
    color_filiere = dico_couleur_filiere[filiere]

    score = pourcentage(data['score'])
    
    fm.Marker(geocode,
    popup=f"<p><strong>{name_tip}</strong></p><p><strong>Secteur&nbsp:&nbsp</strong>{name_grand_secteur}<p>{str(data['PDL'])}&nbsppoint(s)&nbspde&nbsplivraison</p><p><strong>Consommation</strong>:&nbsp{significatifs(data['CONSO'],cs)}&nbspMWh</p><p><em>Fiable&nbspà&nbsp{score}&nbsp%</em></p>",tooltip=name_tip,icon = fm.Icon(color=color_filiere)).add_to(cluster_DLE_all)



In [15]:
m 